In [1]:
!pip install geopandas --upgrade --q
!pip install gtfs_kit --upgrade --q

     |████████████████████████████████| 1.0 MB 34.8 MB/s 
     |████████████████████████████████| 6.3 MB 28.0 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 
     |████████████████████████████████| 62 kB 859 kB/s 
     |████████████████████████████████| 994 kB 54.1 MB/s 
     |████████████████████████████████| 10.0 MB 41.6 MB/s 


In [2]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

DATA_DIR = DIR/'content/'

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# Read feed and describe/content/NorthernKentucky_TANK.zip

path = DATA_DIR/'gtfs (5).zip'
feed = gk.read_feed(path, dist_units='km')
print(feed)

* agency --------------------
	  agency_id  ...                              agency_fare_url
0       466  ...  http://128bc.org/rev-hartwell-area-shuttle/

[1 rows x 7 columns]
* calendar --------------------
	          service_id                         service_name  ...  start_date  end_date
0  c_1725_b_642_d_31  Year-round calendar (Monday-Friday)  ...    20151101  20190101

[1 rows x 11 columns]
* calendar_dates --------------------
	          service_id      date            holiday_name  exception_type
0  c_1725_b_642_d_31  20190101          New Year's Day               2
1  c_1725_b_642_d_31  20181225           Christmas Day               2
2  c_1725_b_642_d_31  20181123  Day after Thanksgiving               2
3  c_1725_b_642_d_31  20181122        Thanksgiving Day               2
4  c_1725_b_642_d_31  20180704        Independence Day               2
* fare_attributes --------------------
	None
* fare_rules --------------------
	None
* feed_info --------------------
	             

In [4]:
feed.describe()

,indicator,value
0,agencies,[128 Business Council]
1,timezone,America/New_York
2,start_date,20151101
3,end_date,20190101
4,num_routes,9
5,num_trips,74
6,num_stops,64
7,num_shapes,44
8,sample_date,20151105
9,num_routes_active_on_sample_date,9


In [5]:
## To check the warnings in the datas
feed.validate()

,type,message,table,rows
0,warning,Unrecognized column service_name,calendar,[]
1,warning,Feed expired,calendar/calendar_dates,[]
2,warning,Unrecognized column holiday_name,calendar_dates,[]
3,warning,Unrecognized column feed_license,feed_info,[]
4,warning,Unrecognized column feed_contact_email,feed_info,[]
5,warning,Unrecognized column feed_contact_url,feed_info,[]
6,warning,Unrecognized column route_sort_order,routes,[]
7,warning,Unrecognized column min_headway_minutes,routes,[]
11,warning,Unrecognized column continuous_stops,stop_times,[]
12,warning,Unrecognized column area_id,stop_times,[]


In [6]:
# List feed
gk.list_feed(path)

,file_name,file_size
0,fare_attributes.txt,81
1,calendar.txt,187
2,feed_info.txt,319
3,farezone_attributes.txt,18
4,areas.txt,12
5,frequencies.txt,53
6,fare_rider_categories.txt,32
7,calendar_dates.txt,544
8,agency.txt,224
9,rider_categories.txt,45


In [7]:
## Compute the trip stats
trip_stats=feed.compute_trip_stats()
trip_stats.head().T

,60,55,23,21,37
trip_id,t_18939_b_642_tn_0,t_18934_b_642_tn_0,t_18902_b_642_tn_0,t_18900_b_642_tn_0,t_18916_b_642_tn_0
route_id,3231,3231,3231,3232,3232
route_short_name,The Rev,The Rev,The Rev,The Rev,The Rev
route_type,3,3,3,3,3
direction_id,NaN,NaN,NaN,NaN,NaN
shape_id,p_6746,p_6760,p_6759,p_6758,p_6745
num_stops,13,14,13,12,14
start_time,06:30:00,07:15:00,08:30:00,16:10:00,16:57:00
end_time,07:15:00,08:30:00,09:40:00,16:57:00,18:25:00
start_stop_id,807645,807693,807693,807691,807645


In [8]:
# Add shape_dist_traveled column to stop times
feed = feed.append_dist_to_stop_times()

In [9]:
feed.stop_times.head().T

,0,1,2,3,4
trip_id,t_18879_b_642_tn_0,t_18879_b_642_tn_0,t_18879_b_642_tn_0,t_18879_b_642_tn_0,t_18879_b_642_tn_0
arrival_time,19:28:00,19:30:00,NaN,NaN,NaN
departure_time,19:28:00,19:30:00,NaN,NaN,NaN
stop_id,835935,835940,835939,835938,835937
stop_sequence,1,2,3,4,5
stop_headsign,NaN,NaN,NaN,NaN,NaN
pickup_type,2,0,2,2,2
drop_off_type,2,0,2,2,2
timepoint,0,0,0,0,0
continuous_stops,0,0,0,0,0


In [10]:
## Choose the study dates 
week=feed.get_first_week()
week

['20151102',
 '20151103',
 '20151104',
 '20151105',
 '20151106',
 '20151107',
 '20151108']

In [11]:
date=[week[0],week[4]] ##  is counted as sunday and 4-wdnesday
date

['20151102', '20151106']

In [12]:
## COmpute the feed time series
fts=feed.compute_feed_time_series(trip_stats,date,freq='2H')

In [13]:
fts

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2015-11-02 00:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2015-11-02 02:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2015-11-02 04:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2015-11-02 06:00:00,6.0,16.0,16.0,343862.847481,11.750000,29264.923190
2015-11-02 08:00:00,25.0,21.0,31.0,536435.252998,19.950000,26888.985113
2015-11-02 10:00:00,8.0,2.0,8.0,80035.151265,2.900000,27598.328022
2015-11-02 12:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2015-11-02 14:00:00,0.0,1.0,1.0,4189.705923,0.283333,14787.197376
2015-11-02 16:00:00,10.0,19.0,20.0,414741.791485,17.200000,24112.894854


In [15]:
gk.downsample(fts,freq='2H')

indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2015-11-02 00:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2015-11-02 02:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2015-11-02 04:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2015-11-02 06:00:00,6.0,16.0,16.0,343862.847481,11.750000,29264.923190
2015-11-02 08:00:00,25.0,21.0,31.0,536435.252998,19.950000,26888.985113
2015-11-02 10:00:00,8.0,2.0,8.0,80035.151265,2.900000,27598.328022
2015-11-02 12:00:00,0.0,0.0,0.0,0.000000,0.000000,0.000000
2015-11-02 14:00:00,0.0,1.0,1.0,4189.705923,0.283333,14787.197376
2015-11-02 16:00:00,10.0,19.0,20.0,414741.791485,17.200000,24112.894854


In [16]:
# Compute feed stats for first week
feed_stats=feed.compute_feed_stats(trip_stats,dates=week)
feed_stats

,num_stops,num_routes,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed,peak_num_trips,peak_start_time,peak_end_time,date
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151102
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151103
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151104
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151105
0,64,9,74,74,74,1.810899e+06,66.316667,27306.842361,11,08:30:00,09:20:00,20151106


In [17]:
# Compute route time series
rts = feed.compute_route_time_series(trip_stats, date, freq='12H')
rts

indicator           num_trip_ends       ... service_speed              
route_id                     3231 3232  ...          4766          4767
datetime                                ...                            
2015-11-02 00:00:00           3.0  0.0  ...  10983.177458  21725.128761
2015-11-02 12:00:00           0.0  3.0  ...   9683.802145  12506.370458
2015-11-03 00:00:00           NaN  NaN  ...           NaN           NaN
2015-11-03 12:00:00           NaN  NaN  ...           NaN           NaN
2015-11-04 00:00:00           NaN  NaN  ...           NaN           NaN
2015-11-04 12:00:00           NaN  NaN  ...           NaN           NaN
2015-11-05 00:00:00           NaN  NaN  ...           NaN           NaN
2015-11-05 12:00:00           NaN  NaN  ...           NaN           NaN
2015-11-06 00:00:00           3.0  0.0  ...  10983.177458  21725.128761
2015-11-06 12:00:00           0.0  3.0  ...   9683.802145  12506.370458

[10 rows x 54 columns]

In [20]:
# Slice time series

inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['3231','3232','4732','4733']

rts.loc[:, (inds, rids)]

indicator           service_distance                 ... service_speed              
route_id                        3231           3232  ...          4732          4733
datetime                                             ...                            
2015-11-02 00:00:00    108390.016441       0.000000  ...  28267.377536  33424.238469
2015-11-02 12:00:00         0.000000  116717.023976  ...  28849.832502  30283.516707
2015-11-03 00:00:00              NaN            NaN  ...           NaN           NaN
2015-11-03 12:00:00              NaN            NaN  ...           NaN           NaN
2015-11-04 00:00:00              NaN            NaN  ...           NaN           NaN
2015-11-04 12:00:00              NaN            NaN  ...           NaN           NaN
2015-11-05 00:00:00              NaN            NaN  ...           NaN           NaN
2015-11-05 12:00:00              NaN            NaN  ...           NaN           NaN
2015-11-06 00:00:00    108390.016441       0.000000  ...  28267.377536  33424.238469
2015-11-06 12:00:00         0.000000  116717.023976  ...  28849.832502  30283.516707

[10 rows x 12 columns]

In [22]:
# Compute trip locations for every hour
rng = pd.date_range('14/12/2018', periods=24, freq='H')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(date[0], times)
loc

,trip_id,time,rel_dist,route_id,direction_id,shape_id,lon,lat
0,t_18880_b_642_tn_0,09:00:00,0.612663,4765,0.0,p_8036,-71.210199,42.296657
1,t_18881_b_642_tn_0,20:00:00,NaN,4732,1.0,p_8022,-71.142620,42.395420
2,t_18882_b_642_tn_0,10:00:00,0.919251,4733,0.0,p_6752,-71.260250,42.390313
3,t_18883_b_642_tn_0,09:00:00,0.000000,4734,0.0,p_6754,-71.142620,42.395420
4,t_18884_b_642_tn_0,07:00:00,0.458245,4732,0.0,p_6748,-71.252567,42.416719
...,...,...,...,...,...,...,...,...
61,t_18949_b_642_tn_0,17:00:00,0.036364,4732,1.0,p_8021,-71.254448,42.405879
62,t_18949_b_642_tn_0,18:00:00,0.878757,4732,1.0,p_8021,-71.233849,42.422961
63,t_18950_b_642_tn_0,09:00:00,0.315956,4732,0.0,p_6748,-71.247798,42.423498
64,t_18951_b_642_tn_0,07:00:00,0.858668,4733,0.0,p_6752,-71.259015,42.393343


In [23]:
feed.routes['route_id'].iat[0]

'4732'

In [50]:
# Build a route timetable
route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, date).T.dropna(axis=1).to_csv('route_time_tabel.csv')

In [25]:
feed.routes.route_id[feed.routes.route_id==route_id]

0    4732
Name: route_id, dtype: object

In [26]:
rids = feed.routes.route_id[feed.routes.route_id=='4732']
feed.map_routes(rids)

/usr/local/lib/python3.7/dist-packages/pandas/core/common.py:471: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  return func(obj, *args, **kwargs)


In [27]:
feed.trips.filter(['route_id','trip_id','shape_id']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73
route_id,3231,3231,3231,3232,3232,3232,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4733,4733,4733,4733,4733,4733,4733,4733,4733,4733,4734,4734,4734,4734,4734,4734,4734,4734,4734,4736,4736,4736,4736,4736,4736,4736,4736,4736,4765,4765,4765,4765,4765,4765,4765,4765,4765,4766,4766,4766,4766,4766,4766,4767,4767,4767,4767,4767
trip_id,t_18902_b_642_tn_0,t_18939_b_642_tn_0,t_18934_b_642_tn_0,t_18888_b_642_tn_0,t_18900_b_642_tn_0,t_18916_b_642_tn_0,t_18952_b_642_tn_0,t_18884_b_642_tn_0,t_18890_b_642_tn_0,t_18907_b_642_tn_0,t_18909_b_642_tn_0,t_18918_b_642_tn_0,t_18919_b_642_tn_0,t_18923_b_642_tn_0,t_18928_b_642_tn_0,t_18950_b_642_tn_0,t_18903_b_642_tn_0,t_18881_b_642_tn_0,t_18949_b_642_tn_0,t_18885_b_642_tn_0,t_18889_b_642_tn_0,t_18908_b_642_tn_0,t_18892_b_642_tn_0,t_18894_b_642_tn_0,t_18896_b_642_tn_0,t_18899_b_642_tn_0,t_18932_b_642_tn_0,t_18882_b_642_tn_0,t_18929_b_642_tn_0,t_18951_b_642_tn_0,t_18941_b_642_tn_0,t_18945_b_642_tn_0,t_18879_b_642_tn_0,t_18931_b_642_tn_0,t_18901_b_642_tn_0,t_18914_b_642_tn_0,t_18883_b_642_tn_0,t_18930_b_642_tn_0,t_18905_b_642_tn_0,t_18895_b_642_tn_0,t_18891_b_642_tn_0,t_18938_b_642_tn_0,t_18911_b_642_tn_0,t_18904_b_642_tn_0,t_18936_b_642_tn_0,t_18948_b_642_tn_0,t_18886_b_642_tn_0,t_18927_b_642_tn_0,t_18935_b_642_tn_0,t_18947_b_642_tn_0,t_18946_b_642_tn_0,t_18887_b_642_tn_0,t_18915_b_642_tn_0,t_18917_b_642_tn_0,t_18913_b_642_tn_0,t_18926_b_642_tn_0,t_18880_b_642_tn_0,t_18897_b_642_tn_0,t_18920_b_642_tn_0,t_18933_b_642_tn_0,t_18893_b_642_tn_0,t_18921_b_642_tn_0,t_18937_b_642_tn_0,t_18925_b_642_tn_0,t_18924_b_642_tn_0,t_18942_b_642_tn_0,t_18922_b_642_tn_0,t_18906_b_642_tn_0,t_18910_b_642_tn_0,t_18943_b_642_tn_0,t_18898_b_642_tn_0,t_18944_b_642_tn_0,t_18940_b_642_tn_0,t_18912_b_642_tn_0
shape_id,p_6759,p_6746,p_6760,p_6745,p_6758,p_6745,p_6750,p_6748,p_6751,p_6751,p_6748,p_6747,p_6747,p_6748,p_6749,p_6748,p_8019,p_8022,p_8021,p_8017,p_8018,p_8020,p_8016,p_8021,p_8019,p_8017,p_6752,p_6752,p_6753,p_6752,p_6753,p_8025,p_8027,p_8026,p_8028,p_8029,p_6754,p_6755,p_6755,p_6755,p_6754,p_8032,p_8031,p_8032,p_8031,p_6756,p_6757,p_6756,p_6757,p_6757,p_8034,p_8033,p_8034,p_8033,p_8036,p_8036,p_8036,p_8036,p_8035,p_8038,p_8038,p_8038,p_8037,p_8040,p_8040,p_8039,p_8024,p_8023,p_8024,p_8042,p_8041,p_8042,p_8044,p_8043


## ## FInd all the trips of specific route : slice rows

In [28]:
## FInd all the trips of specific route : slice rows
feed.trips.loc[lambda x : x.route_id == '3232']

,route_id,service_id,trip_id,trip_short_name,trip_headsign,direction_id,block_id,shape_id,bikes_allowed,wheelchair_accessible,trip_type
3,3232,c_1725_b_642_d_31,t_18888_b_642_tn_0,NaN,NaN,NaN,3271,p_6745,NaN,NaN,NaN
4,3232,c_1725_b_642_d_31,t_18900_b_642_tn_0,NaN,NaN,NaN,3271,p_6758,NaN,NaN,NaN
5,3232,c_1725_b_642_d_31,t_18916_b_642_tn_0,NaN,NaN,NaN,3271,p_6745,NaN,NaN,NaN


In [29]:
## group all the trips by route 
feed.trips.groupby('route_id').agg(['mean']).T


,route_id,3231,3232,4732,4733,4734,4736,4765,4766,4767
direction_id,mean,NaN,NaN,0.5,0.5,0.444444,0.444444,0.444444,0.5,0.4
bikes_allowed,mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wheelchair_accessible,mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trip_type,mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
## Comute the number of trips per route : split ,apply,combine
def my_agg(group):
  d={}
  d['num_trips']=group.shape[0]
  return pd.Series(d)


In [35]:
(feed.trips
 # split 
 .groupby('route_id')
 #apply and combine
 .apply(my_agg))

,num_trips
route_id,
3231,3
3232,3
4732,20
4733,10
4734,9
4736,9
4765,9
4766,6
4767,5


In [36]:
## Use method chaning 
(
  feed.trips
 # split 
 .groupby('route_id')
 #apply and combine
 .apply(my_agg)
 .reset_index()
 #Join in route short name from feeed routes
 .merge(feed.routes.filter(['route_id','route_short_name','route_long_name','route_type']))
 #sort values 
 .sort_values('num_trips',ascending=False).T
)

,2,3,4,5,6,7,8,0,1
route_id,4732,4733,4734,4736,4765,4766,4767,3231,3232
num_trips,20,10,9,9,9,6,5,3,3
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev
route_long_name,Alewife Shuttle Route A,Alewife Shuttle Route B,Alewife Shuttle Route C,Alewife Shuttle Route D,Needham Shuttle,Waltham Shuttle Route A,Waltham Shuttle Route B,Hartwell Area Shuttle (AM),Hartwell Area Shuttle (PM)
route_type,3,3,3,3,3,3,3,3,3


In [37]:
(feed.trips
 # split 
 .groupby('route_id')
 #apply and combine
 .apply(my_agg))

,num_trips
route_id,
3231,3
3232,3
4732,20
4733,10
4734,9
4736,9
4765,9
4766,6
4767,5


In [38]:
feed.trips.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73
route_id,3231,3231,3231,3232,3232,3232,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4732,4733,4733,4733,4733,4733,4733,4733,4733,4733,4733,4734,4734,4734,4734,4734,4734,4734,4734,4734,4736,4736,4736,4736,4736,4736,4736,4736,4736,4765,4765,4765,4765,4765,4765,4765,4765,4765,4766,4766,4766,4766,4766,4766,4767,4767,4767,4767,4767
service_id,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31
trip_id,t_18902_b_642_tn_0,t_18939_b_642_tn_0,t_18934_b_642_tn_0,t_18888_b_642_tn_0,t_18900_b_642_tn_0,t_18916_b_642_tn_0,t_18952_b_642_tn_0,t_18884_b_642_tn_0,t_18890_b_642_tn_0,t_18907_b_642_tn_0,t_18909_b_642_tn_0,t_18918_b_642_tn_0,t_18919_b_642_tn_0,t_18923_b_642_tn_0,t_18928_b_642_tn_0,t_18950_b_642_tn_0,t_18903_b_642_tn_0,t_18881_b_642_tn_0,t_18949_b_642_tn_0,t_18885_b_642_tn_0,t_18889_b_642_tn_0,t_18908_b_642_tn_0,t_18892_b_642_tn_0,t_18894_b_642_tn_0,t_18896_b_642_tn_0,t_18899_b_642_tn_0,t_18932_b_642_tn_0,t_18882_b_642_tn_0,t_18929_b_642_tn_0,t_18951_b_642_tn_0,t_18941_b_642_tn_0,t_18945_b_642_tn_0,t_18879_b_642_tn_0,t_18931_b_642_tn_0,t_18901_b_642_tn_0,t_18914_b_642_tn_0,t_18883_b_642_tn_0,t_18930_b_642_tn_0,t_18905_b_642_tn_0,t_18895_b_642_tn_0,t_18891_b_642_tn_0,t_18938_b_642_tn_0,t_18911_b_642_tn_0,t_18904_b_642_tn_0,t_18936_b_642_tn_0,t_18948_b_642_tn_0,t_18886_b_642_tn_0,t_18927_b_642_tn_0,t_18935_b_642_tn_0,t_18947_b_642_tn_0,t_18946_b_642_tn_0,t_18887_b_642_tn_0,t_18915_b_642_tn_0,t_18917_b_642_tn_0,t_18913_b_642_tn_0,t_18926_b_642_tn_0,t_18880_b_642_tn_0,t_18897_b_642_tn_0,t_18920_b_642_tn_0,t_18933_b_642_tn_0,t_18893_b_642_tn_0,t_18921_b_642_tn_0,t_18937_b_642_tn_0,t_18925_b_642_tn_0,t_18924_b_642_tn_0,t_18942_b_642_tn_0,t_18922_b_642_tn_0,t_18906_b_642_tn_0,t_18910_b_642_tn_0,t_18943_b_642_tn_0,t_18898_b_642_tn_0,t_18944_b_642_tn_0,t_18940_b_642_tn_0,t_18912_b_642_tn_0
trip_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trip_headsign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
direction_id,NaN,NaN,NaN,NaN,

In [40]:
# Compute screen line counts
DATE=week

# Route stats
route_stats = feed.compute_route_stats(trip_stats, dates=DATE)
route_stats.T

,0,1,2,3,4,5,6,7,8,0,1,2,3,4,5,6,7,8,0,1,2,3,4,5,6,7,8,0,1,2,3,4,5,6,7,8,0,1,2,3,4,5,6,7,8
route_id,3231,3232,4732,4733,4734,4736,4765,4766,4767,3231,3232,4732,4733,4734,4736,4765,4766,4767,3231,3232,4732,4733,4734,4736,4765,4766,4767,3231,3232,4732,4733,4734,4736,4765,4766,4767,3231,3232,4732,4733,4734,4736,4765,4766,4767
route_short_name,The Rev,The Rev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,NaN,NaN,NaN,NaN,NaN,NaN,NaN
route_type,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
num_trips,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5
num_trip_starts,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5
num_trip_ends,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5,3,3,20,10,9,9,9,6,5
is_loop,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
is_bidirectional,0,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1
start_time,06:30:00,16:10:00,06:30:00,06:45:00,06:45:00,06:10:00,06:30:00,08:30:00,07:15:00,06:30:00,16:10:00,06:30:00,06:45:00,06:45:00,06:10:00,06:30:00,08:30:00,07:15:00,06:30:00,16:10:00,06:30:00,06:45:00,06:45:00,06:10:00,06:30:00,08:30:00,07:15:00,06:30:00,16:10:00,06:30:00,06:45:00,06:45:00,06:10:00,06:30:00,08:30:00,07:15:00,06:30:00,16:10:00,06:30:00,06:45:00,06:45:00,06:10:00,06:30:00,08:30:00,07:15:00
end_time,09:40:00,19:45:00,20:00:00,19:45:00,19:45:00,19:45:00,18:40:00,18:40:00,19:15:00,09:40:00,19:45:00,20:00:00,19:45:00,19:45:00,19:45:00,18:40:00,18:40:00,19:15:00,09:40:00,19:45:00,20:00:00,19:45:00,19:45:00,19:45:00,18:40:00,18:40:00,19:15:00,09:40:00,19:45:00,20:00:00,19:45:00,19:45:00,19:45:00,18:40:00,18:40:00,19:15:00,09:40:00,19:45:00,20:00:00,19:45:00,19:45:00,19:45:00,18:40:00,18:40:00,19:15:00


In [42]:
feed.get_routes(DATE)

TypeError: ignored

In [43]:
# Join in route long name
route_stats = route_stats.merge(feed.routes.filter(["route_id", "route_long_name"]))

cols = [
    "route_id",
    "route_short_name", 
    "route_long_name", 
    "route_type", 
    "start_time",
    "end_time",
    "num_trips",
    "max_headway",
    "mean_trip_distance",
    "mean_trip_duration",
    "service_speed",
]

In [44]:
# The most/least frequent routes are
(
    route_stats
    .filter(cols)
    .sort_values("max_headway")
    .T
)

,10,11,12,13,14,30,31,32,33,34,36,37,38,39,35,29,28,27,26,15,16,17,19,20,18,22,23,24,25,21,43,40,41,42,44,0,1,2,3,4,5,6,7,8,9
route_id,4732,4732,4732,4732,4732,4765,4765,4765,4765,4765,4766,4766,4766,4766,4766,4736,4736,4736,4736,4733,4733,4733,4733,4734,4733,4734,4734,4734,4736,4734,4767,4767,4767,4767,4767,3231,3231,3231,3231,3231,3232,3232,3232,3232,3232
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev
route_long_name,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route A,Needham Shuttle,Needham Shuttle,Needham Shuttle,Needham Shuttle,Needham Shuttle,Waltham Shuttle Route A,Waltham Shuttle Route A,Waltham Shuttle Route A,Waltham Shuttle Route A,Waltham Shuttle Route A,Alewife Shuttle Route D,Alewife Shuttle Route D,Alewife Shuttle Route D,Alewife Shuttle Route D,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route C,Alewife Shuttle Route B,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route D,Alewife Shuttle Route C,Waltham Shuttle Route B,Waltham Shuttle Route B,Waltham Shuttle Route B,Waltham Shuttle Route B,Waltham Shuttle Route B,Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM)
route_type,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
start_time,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,08:30:00,08:30:00,08:30:00,08:30:00,08:30:00,06:10:00,06:10:00,06:10:00,06:10:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:10:00,06:45:00,07:15:00,07:15:00,07:15:00,07:15:00,07:15:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,16:10:00,16:10:00,16:10:00,16:10:00,16:10:00
end_time,20:00:00,20:00:00,20:00:00,20:00:00,20:00:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:15:00,19:15:00,19:15:00,19:15:00,19:15:00,09:40:00,09:40:00,09:40:00,09:40:00,09:40:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00
num_trips,20,20,20,20,20,9,9,9,9,9,6,6,6,6,6,9,9,9,9,10,10,10,10,9,10,9,9,9,9,9,5,5,5,5,5,3,3,3,3,3,3,3,3,3,3
max_headway,50,50,50,50,50,50,50,50,50,50,70,70,70,70,70,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,110,110,110,110,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_trip_distance,24662.2,24662.2,24662.2,24662.2,24662.2,11142.3,11142.3,11142.3,11142.3,11142.3,8288.11,8288.11,8288.11,8288.11,8288.11,33684.7,33684.7,33684.7,33684.7,27317.3,27317.3,27317.3,27317.3,31069.7,27317.3,31069.7,31069.7,31069.7,33684.7,31069.7,17315.4,17315.4,17315.4,17315.4,17315.4,36130,36130,36130,36130,36130,38905.7,38905.7,38905.7,38905.7,38905.7
mean_trip_duration,0.864167,0.864167,0.864167,0.864167,0.864167,0.696296,0.696296,0.696296,0.696296,0.696296,0.805556,0.805556,0.805556,0.805556,0.805556,0.888889,0.888889,0.888889,0.888889,0.863333,0.863333,0.863333,0.863333,1.05185,0.863333,1.05185,1.05185,1.05185,0.888889,1.05185,1.02333,1.02333,1.02333,1.02333,1.02333,1.05556,1.05556,1.05556,1.05556,1.05556,1.18333,1.18333,1.18333,1.18333,1.18333


In [45]:
# The shortest/longest routes are
(
    route_stats
    .filter(cols)
    .sort_values("mean_trip_distance")
    .T
)

,39,38,37,36,35,30,32,33,34,31,43,44,40,41,42,10,13,14,11,12,15,16,17,18,19,22,24,23,21,20,26,27,28,29,25,1,2,3,4,0,6,7,9,5,8
route_id,4766,4766,4766,4766,4766,4765,4765,4765,4765,4765,4767,4767,4767,4767,4767,4732,4732,4732,4732,4732,4733,4733,4733,4733,4733,4734,4734,4734,4734,4734,4736,4736,4736,4736,4736,3231,3231,3231,3231,3231,3232,3232,3232,3232,3232
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev
route_long_name,Waltham Shuttle Route A,Waltham Shuttle Route A,Waltham Shuttle Route A,Waltham Shuttle Route A,Waltham Shuttle Route A,Needham Shuttle,Needham Shuttle,Needham Shuttle,Needham Shuttle,Needham Shuttle,Waltham Shuttle Route B,Waltham Shuttle Route B,Waltham Shuttle Route B,Waltham Shuttle Route B,Waltham Shuttle Route B,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route D,Alewife Shuttle Route D,Alewife Shuttle Route D,Alewife Shuttle Route D,Alewife Shuttle Route D,Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM)
route_type,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
start_time,08:30:00,08:30:00,08:30:00,08:30:00,08:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,07:15:00,07:15:00,07:15:00,07:15:00,07:15:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:10:00,06:10:00,06:10:00,06:10:00,06:10:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,16:10:00,16:10:00,16:10:00,16:10:00,16:10:00
end_time,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,19:15:00,19:15:00,19:15:00,19:15:00,19:15:00,20:00:00,20:00:00,20:00:00,20:00:00,20:00:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,09:40:00,09:40:00,09:40:00,09:40:00,09:40:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00
num_trips,6,6,6,6,6,9,9,9,9,9,5,5,5,5,5,20,20,20,20,20,10,10,10,10,10,9,9,9,9,9,9,9,9,9,9,3,3,3,3,3,3,3,3,3,3
max_headway,70,70,70,70,70,50,50,50,50,50,110,110,110,110,110,50,50,50,50,50,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_trip_distance,8288.11,8288.11,8288.11,8288.11,8288.11,11142.3,11142.3,11142.3,11142.3,11142.3,17315.4,17315.4,17315.4,17315.4,17315.4,24662.2,24662.2,24662.2,24662.2,24662.2,27317.3,27317.3,27317.3,27317.3,27317.3,31069.7,31069.7,31069.7,31069.7,31069.7,33684.7,33684.7,33684.7,33684.7,33684.7,36130,36130,36130,36130,36130,38905.7,38905.7,38905.7,38905.7,38905.7
mean_trip_duration,0.805556,0.805556,0.805556,0.805556,0.805556,0.696296,0.696296,0.696296,0.696296,0.696296,1.02333,1.02333,1.02333,1.02333,1.02333,0.864167,0.864167,0.864167,0.864167,0.864167,0.863333,0.863333,0.863333,0.863333,0.863333,1.05185,1.05185,1.05185,1.05185,1.05185,0.888889,0.888889,0.888889,0.888889,0.888889,1.05556,1.05556,1.05556,1.05556,1.05556,1.18333,1.18333,1.18333,1.18333,1.18333


In [46]:
# The slowest/fastest routes are
(
    route_stats
    .filter(cols)
    .sort_values("service_speed")
    .T
)

,39,38,37,36,35,30,31,32,34,33,44,43,40,41,42,12,13,14,11,10,23,24,22,20,21,16,17,18,15,19,9,8,7,6,5,4,3,2,1,0,29,27,26,25,28
route_id,4766,4766,4766,4766,4766,4765,4765,4765,4765,4765,4767,4767,4767,4767,4767,4732,4732,4732,4732,4732,4734,4734,4734,4734,4734,4733,4733,4733,4733,4733,3232,3232,3232,3232,3232,3231,3231,3231,3231,3231,4736,4736,4736,4736,4736
route_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,The Rev,NaN,NaN,NaN,NaN,NaN
route_long_name,Waltham Shuttle Route A,Waltham Shuttle Route A,Waltham Shuttle Route A,Waltham Shuttle Route A,Waltham Shuttle Route A,Needham Shuttle,Needham Shuttle,Needham Shuttle,Needham Shuttle,Needham Shuttle,Waltham Shuttle Route B,Waltham Shuttle Route B,Waltham Shuttle Route B,Waltham Shuttle Route B,Waltham Shuttle Route B,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route A,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route C,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route B,Alewife Shuttle Route B,Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (PM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Hartwell Area Shuttle (AM),Alewife Shuttle Route D,Alewife Shuttle Route D,Alewife Shuttle Route D,Alewife Shuttle Route D,Alewife Shuttle Route D
route_type,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
start_time,08:30:00,08:30:00,08:30:00,08:30:00,08:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,07:15:00,07:15:00,07:15:00,07:15:00,07:15:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,06:45:00,16:10:00,16:10:00,16:10:00,16:10:00,16:10:00,06:30:00,06:30:00,06:30:00,06:30:00,06:30:00,06:10:00,06:10:00,06:10:00,06:10:00,06:10:00
end_time,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,18:40:00,19:15:00,19:15:00,19:15:00,19:15:00,19:15:00,20:00:00,20:00:00,20:00:00,20:00:00,20:00:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00,09:40:00,09:40:00,09:40:00,09:40:00,09:40:00,19:45:00,19:45:00,19:45:00,19:45:00,19:45:00
num_trips,6,6,6,6,6,9,9,9,9,9,5,5,5,5,5,20,20,20,20,20,9,9,9,9,9,10,10,10,10,10,3,3,3,3,3,3,3,3,3,3,9,9,9,9,9
max_headway,70,70,70,70,70,50,50,50,50,50,110,110,110,110,110,50,50,50,50,50,80,80,80,80,80,80,80,80,80,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80,80,80,80,80
mean_trip_distance,8288.11,8288.11,8288.11,8288.11,8288.11,11142.3,11142.3,11142.3,11142.3,11142.3,17315.4,17315.4,17315.4,17315.4,17315.4,24662.2,24662.2,24662.2,24662.2,24662.2,31069.7,31069.7,31069.7,31069.7,31069.7,27317.3,27317.3,27317.3,27317.3,27317.3,38905.7,38905.7,38905.7,38905.7,38905.7,36130,36130,36130,36130,36130,33684.7,33684.7,33684.7,33684.7,33684.7
mean_trip_duration,0.805556,0.805556,0.805556,0.805556,0.805556,0.696296,0.696296,0.696296,0.696296,0.696296,1.02333,1.02333,1.02333,1.02333,1.02333,0.864167,0.864167,0.864167,0.864167,0.864167,1.05185,1.05185,1.05185,1.05185,1.05185,0.863333,0.863333,0.863333,0.863333,0.863333,1.18333,1.18333,1.18333,1.18333,1.18333,1.05556,1.05556,1.05556,1.05556,1.05556,0.888889,0.888889,0.888889,0.888889,0.888889


In [48]:
feed.geometrize_routes()

/usr/local/lib/python3.7/dist-packages/pandas/core/common.py:471: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  return func(obj, *args, **kwargs)


,route_id,geometry,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order,min_headway_minutes
0,3231,"LINESTRING (-71.14262 42.39544, -71.14267 42.3...",466,The Rev,Hartwell Area Shuttle (AM),NaN,3,NaN,400080,ffffff,5,NaN
1,3232,"MULTILINESTRING ((-71.26355 42.48266, -71.2637...",466,The Rev,Hartwell Area Shuttle (PM),NaN,3,NaN,400080,ffffff,6,NaN
2,4732,"MULTILINESTRING ((-71.14262 42.39542, -71.1426...",466,NaN,Alewife Shuttle Route A,NaN,3,http://128bc.org/alewife-shuttle-route-a/,ff0000,ffffff,1,NaN
3,4733,"MULTILINESTRING ((-71.26008 42.39566, -71.2599...",466,NaN,Alewife Shuttle Route B,NaN,3,http://128bc.org/alewife-shuttle-route-b/,ff0013,ffffff,2,NaN
4,4734,"MULTILINESTRING ((-71.14262 42.39542, -71.1426...",466,NaN,Alewife Shuttle Route C,NaN,3,http://128bc.org/alewife-shuttle-route-c/,d21f14,ffffff,3,NaN
5,4736,"MULTILINESTRING ((-71.27690 42.40966, -71.2769...",466,NaN,Alewife Shuttle Route D,NaN,3,http://128bc.org/alewife-shuttle-route-d/,e9001e,ffffff,4,NaN
6,4765,"MULTILINESTRING ((-71.21482 42.29450, -71.2149...",466,NaN,Needham Shuttle,NaN,3,http://128bc.org/needham-shuttle/,005929,ffffff,7,NaN
7,4766,"MULTILINESTRING ((-71.23572 42.37455, -71.2356...",466,NaN,Waltham Shuttle Route A,NaN,3,http://128bc.org/Waltham_route_A,96007e,ffffff,8,NaN
8,4767,"MULTILINESTRING ((-71.27546 42.40682, -71.2753...",466,NaN,Waltham Shuttle Route B,NaN,3,http://128bc.org/waltham-shuttle/,96007e,ffffff,9,NaN


In [68]:
df_stops=feed.geometrize_stops()

In [69]:
df_routes=feed.geometrize_routes()

/usr/local/lib/python3.7/dist-packages/pandas/core/common.py:471: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  return func(obj, *args, **kwargs)


In [91]:
rids

['4732', '4733', '4734', '4736', '3231', '3232', '4765', '4766', '4767']

In [101]:
feed.build_route_timetable(route_id='3232',dates=feed.get_first_week()).T

,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,40,41,42,43,44,45,46,47,48,49,50,51,52,53,...,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,40,41,42,43,44,45,46,47,48,49,50,51,52,53
route_id,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,...,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232,3232
service_id,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,...,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31
trip_id,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,...,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18900_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18916_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0,t_18888_b_642_tn_0
trip_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trip_headsign,N

In [90]:
df=feed.build_route_timetable(route_id=ids,dates=feed.get_first_week())
df=df.merge(df_routes,on='route_id')
df=df.merge(df_stops,on='stop_id')
df=df[imp_cols].head(40)
df.to_csv(ids+str('.csv'))


,670,671,672,667,668,669,673,674,675,676,677,680,681,682,678,679,670,671,672,667,668,669,673,674,675,676,677,680,681,682,678,679,670,671,672,667,668,669,673,674,675,676,677,680,681,682,678,679,670,671,672,667,668,669,673,674,675,676,677,680,681,682,678,679,670,671,672,667,668,669,673,674,675,676,677,680,681,682,678,679
route_id,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766,4766
service_id,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31,c_1725_b_642_d_31
trip_id,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18942_b_642_tn_0,t_18942_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18906_b_642_tn_0,t_18906_b_642_tn_0,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18942_b_642_tn_0,t_18942_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18906_b_642_tn_0,t_18906_b_642_tn_0,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18942_b_642_tn_0,t_18942_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18906_b_642_tn_0,t_18906_b_642_tn_0,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18942_b_642_tn_0,t_18942_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18906_b_642_tn_0,t_18906_b_642_tn_0,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18924_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18925_b_642_tn_0,t_18942_b_642_tn_0,t_18942_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18922_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18910_b_642_tn_0,t_18906_b_642_tn_0,t_18906_b_642_tn_0
trip_short_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [73]:
df_route_with=df_rtt.merge(df_routes,on='route_id')

In [75]:
df_route_with.merge(df_stops,on='stop_id').columns

Index(['route_id', 'service_id', 'trip_id', 'trip_short_name', 'trip_headsign',
       'direction_id', 'block_id', 'shape_id', 'bikes_allowed',
       'wheelchair_accessible', 'trip_type', 'arrival_time', 'departure_time',
       'stop_id', 'stop_sequence', 'stop_headsign', 'pickup_type',
       'drop_off_type', 'timepoint', 'continuous_stops', 'area_id',
       'service_area_radius', 'shape_dist_traveled', 'date', 'geometry_x',
       'agency_id', 'route_short_name', 'route_long_name', 'route_desc',
       'route_type', 'route_url', 'route_color', 'route_text_color',
       'route_sort_order', 'min_headway_minutes', 'stop_code', 'platform_code',
       'stop_name', 'stop_desc', 'zone_id', 'stop_url', 'location_type',
       'parent_station', 'stop_timezone', 'position', 'direction',
       'wheelchair_boarding', 'geometry_y'],
      dtype='object')

In [93]:
imp_cols=['route_id','stop_id', 'stop_sequence', 'shape_dist_traveled', 'date',
       'route_short_name', 'route_long_name','min_headway_minutes','stop_name', 'stop_desc', 'geometry_y']

In [100]:
for ids in rids:
  df=(ids+str('df'))
  df=feed.build_route_timetable(route_id=ids,dates=feed.get_first_week())
  df1=df.merge(df_routes,on='route_id')
  df2=df1.merge(df_stops,on='stop_id')
  df2=df2[imp_cols]
  df2.drop_duplicates(subset='stop_sequence',keep='first').sort_values('stop_sequence').to_csv(ids+str('.csv'))

  

  

In [ ]:
df.join()

In [80]:
def trim_to_imp_table(rtt_df):
  df=rtt_df.join(df_routes,on='route_id')
  df1=df.join(df_stops,on='stop_id')
  df1=df1[imp_cols]
  df1.to_csv(rtt_df+str('.csv'))


In [55]:
rids=feed.routes.route_id.tolist()

In [47]:
# Map routes

# Map our extreme routes
feed.map_routes(route_ids=feed.routes.route_id)


/usr/local/lib/python3.7/dist-packages/pandas/core/common.py:471: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  return func(obj, *args, **kwargs)


In [105]:
feed.stops_to_geojson()

{'features': [{'geometry': {'coordinates': [-71.221793, 42.299865000000004],
    'type': 'Point'},
   'properties': {'direction': None,
    'location_type': 0,
    'parent_station': None,
    'platform_code': None,
    'position': None,
    'stop_code': None,
    'stop_desc': 'Bulfinch',
    'stop_id': '836589',
    'stop_name': '250 First Ave',
    'stop_timezone': 'America/New_York',
    'stop_url': None,
    'wheelchair_boarding': 0,
    'zone_id': None},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-71.27481973263359, 42.4073471468487],
    'type': 'Point'},
   'properties': {'direction': None,
    'location_type': 0,
    'parent_station': None,
    'platform_code': None,
    'position': None,
    'stop_code': None,
    'stop_desc': 'Bay Colony Corporate Center',
    'stop_id': '835970',
    'stop_name': '1000 Winter Street',
    'stop_timezone': 'America/New_York',
    'stop_url': None,
    'wheelchair_boarding': 0,
    'zone_id': None},
   'type': 'Feature'},
  {'geomet

In [106]:
import json

In [107]:
with open('data.json', 'w') as fp:
    json.dump(feed.stops_to_geojson(), fp)
